In [4]:
import pandas as pd
import os 
os.chdir(r'C:\Users\Arne\Documents\DataScience\Personalisation-for-Media\assignment2')
movies = pd.read_csv(r'data\movies.csv', low_memory=False)
ratings = pd.read_csv(r'data\ratings.csv', low_memory=False)
movie_ratings = pd.merge(movies, ratings)
movie_ratings.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
# additional column for watchtime if user rating is absent after watching a movie
movie_ratings['watch_duration'] = ''

In [6]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp,watch_duration
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,


In [2]:
from itertools import permutations

# create a function that makes combinations of the movies a user (x) reviewed
def create_combinations(x):
    combinations = pd.DataFrame(list(permutations(x.values, 2)), columns=['movie_a', 'movie_b'])
    return combinations

In [3]:
# search for users that have more than a certain amount of reviews
avg_user = movie_ratings.groupby(['userId']).size().reset_index(name='counts')
avg_user = avg_user.set_index('userId')
avg_user = avg_user[avg_user['counts'] > 20]
avg_user_idx = avg_user.index

# make a smaller selection
smaller_selection = movie_ratings.iloc[avg_user_idx]
smaller_selection.head()

,movieId,title,genres,userId,rating,timestamp
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,18,3.5,1455209816


In [4]:
smaller_selection.shape

(596, 6)

In [5]:
# use the create_combinations function
movie_combinations = smaller_selection.groupby('userId')['title'].apply(create_combinations)
movie_combinations.head()

movie_a                             movie_b
userId                                                               
1      0  Grumpier Old Men (1995)                         Heat (1995)
       1              Heat (1995)             Grumpier Old Men (1995)
6      0           Jumanji (1995)             Grumpier Old Men (1995)
       1           Jumanji (1995)            Waiting to Exhale (1995)
       2           Jumanji (1995)  Father of the Bride Part II (1995)

In [6]:
# calculate the occurences of movie a and movie
movie_combinations = movie_combinations.reset_index(drop=True)
combi_count = movie_combinations.groupby(['movie_a', 'movie_b']).size()
combi_count = combi_count.to_frame(name='size').reset_index()
combi_count

,movie_a,movie_b,size
0,Father of the Bride Part II (1995),Grumpier Old Men (1995),18
1,Father of the Bride Part II (1995),Heat (1995),22
2,Father of the Bride Part II (1995),Jumanji (1995),20
3,Father of the Bride Part II (1995),Sabrina (1995),20
4,Father of the Bride Part II (1995),Sudden Death (1995),7
...,...,...,...
63,Waiting to Exhale (1995),Heat (1995),2
64,Waiting to Exhale (1995),Jumanji (1995),2
65,Waiting to Exhale (1995),Sabrina (1995),6
66,Waiting to Exhale (1995),Tom and Huck (1995),1


In [7]:
# sort
combi_count = combi_count.sort_values('size', ascending=False)
combi_count.head(10)

,movie_a,movie_b,size
30,Jumanji (1995),Toy Story (1995),63
56,Toy Story (1995),Jumanji (1995),63
22,Heat (1995),Toy Story (1995),54
55,Toy Story (1995),Heat (1995),54
18,Heat (1995),Jumanji (1995),36
26,Jumanji (1995),Heat (1995),36
14,Grumpier Old Men (1995),Toy Story (1995),31
54,Toy Story (1995),Grumpier Old Men (1995),31
53,Toy Story (1995),Father of the Bride Part II (1995),30
38,Sabrina (1995),Toy Story (1995),30


In [8]:
# search for a specific movie
combi_count[combi_count['movie_a'] == 'Toy Story (1995)'].head(10)

,movie_a,movie_b,size
56,Toy Story (1995),Jumanji (1995),63
55,Toy Story (1995),Heat (1995),54
54,Toy Story (1995),Grumpier Old Men (1995),31
53,Toy Story (1995),Father of the Bride Part II (1995),30
57,Toy Story (1995),Sabrina (1995),30
58,Toy Story (1995),Sudden Death (1995),10
59,Toy Story (1995),Tom and Huck (1995),4
60,Toy Story (1995),Waiting to Exhale (1995),2
